In [462]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [463]:
def get_html(url):
    site = requests.get(url)
    html = BeautifulSoup(site.text, 'html.parser')
    return html

<h2>Trying to parse Wrike's community

In [13]:
HT_root = 'https://help.wrike.com/hc/en-us/community/topics/200428775-How-To'
html = get_html(HT_root)

In [168]:
def get_lis(html):
    cont = html.find('div', class_='forum topic-page container-small')
    topics_ul = cont.find('ul')
    lis = topics_ul.find_all('li')
    return lis[::3]

In [169]:
clear_lis = get_lis(html)

In [170]:
need_symbols = [" ", "-", "'"]

In [173]:
cols = ['url', 'title', 'clear_title']#, 'question', 'answer', 'author']
data = pd.DataFrame(columns=cols)
for li in clear_lis:
    di = li.find('div')
    sp = di.find('span')
    a = sp.find('a')
    title = a.get('title')
    
    #Clear titles from trash symbols (including numbers) and converts to lowercase
    clear_title = []
    need_symbols = [" ", "-", "’"]
    title = title.lower()
    for char in title:
        if((ord(char) >= 97 and ord(char) <= 122) or char in need_symbols):
            clear_title.append(char)
    clear_title = ''.join(clear_title)
    
    data = data.append({
        'url' : a.get('href'), 
        'title' : title, 
        'clear_title' : clear_title
        #'question': , 
        #'answer' : , 
        #'author':
    }, ignore_index=True)

In [174]:
data

,url,title,clear_title
0,https://help.wrike.com/hc/en-us/community/post...,[read me] workspace acting strange? here’s wha...,read me workspace acting strange here’s what t...
1,https://help.wrike.com/hc/en-us/community/post...,[quick question!] should you be posting in onb...,quick question should you be posting in onboar...
2,https://help.wrike.com/hc/en-us/community/post...,blueprints,blueprints
3,https://help.wrike.com/hc/en-us/community/post...,wrike does not work on ios 13.3,wrike does not work on ios
4,https://help.wrike.com/hc/en-us/community/post...,how do i remove a folder that was shared with me?,how do i remove a folder that was shared with me
5,https://help.wrike.com/hc/en-us/community/post...,mandatory custom fields when completing task,mandatory custom fields when completing task
6,https://help.wrike.com/hc/en-us/community/post...,wrike analyze - custom field value sum and bre...,wrike analyze - custom field value sum and bre...
7,https://help.wrike.com/hc/en-us/community/post...,communicate within the team,communicate within the team
8,https://help.wrike.com/hc/en-us/community/post...,copy/move attached file (with comments) to ano...,copymove attached file with comments to anothe...
9,https://help.wrike.com/hc/en-us/community/post...,how to get custom field to display in report,how to get custom field to display in report


In [166]:
url1 = 'https://help.wrike.com/hc/en-us/community/posts/360048928974-How-to-get-Custom-Field-to-display-in-Report'

In [327]:
def clear_comments(comment):
    #Clear questions and answers from introductory sentences (paragraphs)
    trash_words = ['hi ', 'hello', 'hey', 'let me know', 'thanks', 'any help']
    clear = []
    raw = []
    for c in comment:
        c = str(c.contents[0])
        c = c.lower()
        if not any(word in c for word in trash_words):
            clear.append(c)
        else:
            raw.append('(EXCLUDED)')
        raw.append(c + '(new line)')
    return ' '.join(raw) , ' '.join(clear) 

In [345]:
def get_QandA(url):
    html = get_html(url)
    
    #Find question
    post = html.find('div', class_='post')
    question = post.find('article')
    question = question.find('div', class_='post-body')
    
    question = question.find_all('p')
    print('Question:')
    #for q in question:
        #print(q.contents[0])
    raw_q, clear_q = clear_comments(question)
    print('Raw: ', raw_q, '\n')
    print('Clear: ', clear_q, '\n')
    
    #Find 1st comment (usually answer)
    comments = post.find('ul', id='comments')
    c1 = comments.find('section', class_='comment-body')
    
    answer = c1.find_all('p')
    #print("Raw:", answer)
    
    #Find signature (author's  status: user or member of Wrike team)
    if(len(answer[-1].find_all('span')) != 0):
        sign = answer[-1].find_all('span')[1]
        sign = sign.contents[0]
        answer = answer[:-1]
    else:
        sign = 'User'
    print('Answer:')
    #for a in answer:
        #print(a.contents[0])
    raw_a, clear_a = clear_comments(answer)
    print('Raw: ', raw_a, '\n')
    print('Clear: ', clear_a, '\n')
    print('Author: ', sign, '\n')

In [346]:
for url in data.iloc[2:]['url'].values:
    print('Title: ', data.iloc[data.index[data['url'] == url].tolist()[0]]['title'], '\n')
    get_QandA(url)

Title:  blueprints 

Question:
Raw:   

Clear:   

Answer:
Raw:  hey (new line) you could copy/paste the new tasks from the blueprint into the project and delete the older tasks. this way, you want to delete the project.(new line) (EXCLUDED) let me know if this helps! if you need anything else, be sure to let me know.(new line) 

Clear:  hey  you could copy/paste the new tasks from the blueprint into the project and delete the older tasks. this way, you want to delete the project. 

Author:  Community Team at Wrike 

Title:  wrike does not work on ios 13.3 

Question:
Raw:  (EXCLUDED) hi team, my wrike application on iphone has worked fluently and perfectly until i update my iphone software to 13.3 version. i have reinstalled and installed many times, but it has not worked. sometimes, i am able to use 1-2 minutes, then this is exited unexpectedly. please help to support me. thank you!!(new line) 

Clear:   

Answer:
Raw:  hey (new line) (EXCLUDED) i've just raised a support ticket for 

Question:
Raw:  (EXCLUDED) hello,(new line) i want to assign a task to appear on a specific date on my team's inbox. is there a way to do it.(new line)  (new line) 

Clear:  i want to assign a task to appear on a specific date on my team's inbox. is there a way to do it.   

Answer:
Raw:  (EXCLUDED) hi (new line) when you assign a task to a team member, they will receive a notification about it the day before it's due. they can also choose to snooze notifications until a certain date by clicking the clock icon that appears beside the notification in the inbox. this way, they won't be reminded of the task again until that date.(new line) (EXCLUDED) hope this helps! if you have any other questions, be sure to let me know.(new line) 

Clear:  when you assign a task to a team member, they will receive a notification about it the day before it's due. they can also choose to snooze notifications until a certain date by clicking the clock icon that appears beside the notification in the inbox

Question:
Raw:  i've finally gotten around to trying the attach website link feature and it seems utterly useless, as it won't allow me to attach links to pages on our staging server, only our public-facing production server. it returns the error "website can't be accessed" when i use a private url from our staging server. as a proofing tool, it doesn't help me to review pages that are already on the production server. (new line) is anyone else having this problem? is there anything that i can do force wrike to let me enter the url of my choice, even if it wrongly thinks there's no page there? (new line) 

Clear:  i've finally gotten around to trying the attach website link feature and it seems utterly useless, as it won't allow me to attach links to pages on our staging server, only our public-facing production server. it returns the error "website can't be accessed" when i use a private url from our staging server. as a proofing tool, it doesn't help me to review pages that are alrea

Question:
Raw:  can you add a list to a board view so we could have a on hold status or a backlog/pending list.  i suppose that's the same as saying can we add more statuses to a task(new line) 

Clear:  can you add a list to a board view so we could have a on hold status or a backlog/pending list.  i suppose that's the same as saying can we add more statuses to a task 

Answer:
Raw:  hey (new line) it currently isn't possible to assign two statuses to the same task in wrike. as a workaround for the board view, you could add a custom field to your tasks to describe the status of each of your tasks, and then choose to have it shown in the board view. this way, you can mark tasks as a backlog without having to change it's status.(new line) (EXCLUDED) hope this helped! if you have any other questions, be sure to let me know.(new line) 

Clear:  hey  it currently isn't possible to assign two statuses to the same task in wrike. as a workaround for the board view, you could add a custom fiel

<h4>Conclusion: very dirty data itself, it's need to check all the comments (answers on question) manually